## Julia in Data Modeling and Exploration

In [ ]:
using CSVFiles # loading/saving csv
using DataFrames
using DataFramesMeta
using Dates
using RCall
using PyCall
using VegaDatasets
using Statistics
using DataVoyager
using VegaDatasets
ENV["COLUMNS"]=1000; # for dataframe column size

In [ ]:
top5(x::DataFrame) = first(x,5)
last5(x::DataFrame) = last(x,5)

Let's use the `cars` dataset from VegaDatasets for modeling 

In [ ]:
cars = dataset("cars") |> DataFrame
cars |> top5

## Data Voyager

Let's feed the `cars` dataset to the Voyager visualization tool and perform some exploratory visualization.

Note: This won't work in remote notebooks.

In [ ]:
# cars |> Voyager()

## Julia SQL-like data processing workflow

## High-level queries to manipulate data

### Filter rows

In [ ]:
@linq cars |> 
   where(:Origin .== "USA") |> 
   top5

In [ ]:
@subset(cars,:Origin .== "USA") |> last5

### Filter rows and columns

In [ ]:
@linq cars |> 
      where(:Origin .== "Europe") |> # filter rows
      select(:Name,:Year) |>         # subset of columns
      top5

### Grouping, Aggregation, and Sorting

In [ ]:
mymean(x)=mean(skipmissing(x))

In [ ]:
@linq cars |> 
      by(:Origin,
          meanAcc = mymean(:Acceleration),
          meanDisp = mymean(:Displacement),
          meanMPG=mymean(:Miles_per_Gallon)
      ) 

In [ ]:
region_stats = @linq  cars |>
                      by([:Origin,:Cylinders],
                          MeanAcceleration=mymean(:Acceleration),
                          MeanHorsepower=mymean(:Horsepower)) |>
                      orderby(:Origin,:Cylinders)

In [ ]:
res=combine(groupby(cars,[:Origin,:Cylinders])) do x
    macc=mymean(x.Acceleration)
    mincy=mymean(x.Horsepower)
    DataFrame(MeanAcceleration=macc,MeanHorsepower=mincy)
end

In [ ]:
sort!(res,[:Origin,:Cylinders])

## R SQL-like data processing workflow

In [ ]:
R"""
library(tidyverse)
library(ggplot2)
"""

In [ ]:
rcars = cars
@rput rcars; # copy rcars to R environment

In [ ]:
# rcars is visible in R
R"""
rcars %>%
  filter(Origin == "USA") %>%
  select(Origin,Cylinders,Horsepower) %>%
  head
"""

In [ ]:
R"""
results = rcars %>%
          group_by(Origin,Cylinders) %>%
          summarise(
               MeanAcceleration=mean(Acceleration),
               MinHorsepower=min(Horsepower)
          )
"""

In [ ]:
@rget results # copy results to Julia environment

## Data Visualization

In [ ]:
R"""
library(ggplot2)

ggplot(rcars,aes(Horsepower,Acceleration)) +
   geom_point(aes(color=as.factor(Cylinders))) + 
   stat_smooth(method='lm')+
   facet_wrap( ~ Origin) 
"""

In [ ]:
R"names(rcars)"

In [ ]:
R"""
library(ggplot2)

ggplot(rcars,aes(Horsepower,Miles_per_Gallon)) +
   geom_point(aes(color=as.factor(Cylinders))) +
   stat_smooth(method='auto')
"""

In [ ]:
R"""
library(ggplot2)

ggplot(rcars,aes(Displacement,Miles_per_Gallon)) +
   geom_point(aes(color=as.factor(Cylinders))) + 
   stat_smooth()
"""

In [ ]:
R"""
library(ggplot2)

ggplot(rcars,aes(Weight_in_lbs,Miles_per_Gallon)) +
   geom_point(aes(color=as.factor(Cylinders))) +
   stat_smooth()
"""

In [ ]:
sort(cars,:Acceleration) |> top5

In [ ]:
R"""
library(randomForest)

ccars=rcars[complete.cases(rcars),]

rf_model = randomForest(Miles_per_Gallon ~ Cylinders + Displacement + 
                     Horsepower + Acceleration + Weight_in_lbs, data=ccars)
"""

In [ ]:
R"varImpPlot(rf_model)"

In [ ]:
R"""

ccars=rcars[complete.cases(rcars),]

lm_model = lm(Miles_per_Gallon ~ Cylinders + Displacement + 
              Horsepower + Acceleration + Weight_in_lbs, data=ccars)
"""

In [ ]:
R"summary(lm_model)"

In [ ]:
R"""
library(caret)

ccars=rcars[complete.cases(rcars),]

crt_model = train(Miles_per_Gallon ~ Cylinders + Displacement + 
                     Horsepower + Acceleration + Weight_in_lbs, data=ccars,method='rf')
"""

In [ ]:
R"crt_model$finalModel"

In [ ]:
R"varImpPlot(crt_model$finalModel)"